In [3]:
from openai import OpenAI
import json
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()


name = "gpt-3.5-turbo memory-1 user"
model = "gpt-3.5-turbo"               # important options: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
n = 2
memory = 1
stable_system_message = False
assistant_transform = True    
assistant_transform_type = "user"     # options: "user", "user-assistant", "system"

first_prompt = [
  #{"role": "system", "content": ""},
  {"role": "user", "content": ""}
]

def create_completions(name,n,memory,stable_system_message,assistant_transform,assistant_transform_type,first_prompt):

  for iteration in range(n):
    # Load completions from json, if continuing with a file generated in the past

    try:
      with open(f"{name}.json", "r") as json_file:
        completions=json.load(json_file)

    # If not, create a new completions database

    except FileNotFoundError:
      completions = []

    # Collect messages generated so far

    messages = first_prompt
    for completion in completions:
        message = completion["choices"][0]["message"]
        messages.append({"role": message["role"], "content": message["content"]}   
        )

    # Pick prompt messages

    if memory >= len(messages):
        prompt_messages = messages
    else:
        #print(f"Memory smaller than the entire conversation. Forgetting {len(messages)-memory} messages")
        prompt_messages = []
        if stable_system_message:
          prompt_messages.append(first_prompt[0])
          print(prompt_messages)
          print("stable_system_message")
        prompt_messages = prompt_messages+messages[(len(messages)-memory):]

    # Transform assistant role to assistant and user, user, or system roles

    if assistant_transform:
      if assistant_transform_type == "user-assistant":
        i = 1
        while i <= len(prompt_messages):
          if prompt_messages[-i]["role"] == "assistant":
            prompt_messages[-i]["role"] = "user"
          i += 2
      else:
        for prompt_message in prompt_messages:
          if prompt_message["role"] == "assistant":
            prompt_message["role"] = assistant_transform_type


    # Send to OpenAI API to get a new completion

    completion = client.chat.completions.create(
          model=model,
          messages = prompt_messages
        )
    print(f"Prompt messages sent to OpenAI API:{prompt_messages}")

    # Update database and save it to the right json file

    completions.append(completion.model_dump())
    name = f"{model} memory-{memory} {assistant_transform_type}"
    with open(f"{name}.json", "w") as json_file:
        json.dump(completions,json_file)

create_completions(name,n,memory,stable_system_message,assistant_transform,assistant_transform_type,first_prompt)


## Display data

# Load the JSON file

file_path = f"{name}.json"
with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the JSON data in completions

normalized_data = pd.json_normalize(data, record_path='choices', meta=[
    'id', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 
    ['usage', 'completion_tokens'], ['usage', 'prompt_tokens'], ['usage', 'total_tokens']], sep='.',
    meta_prefix='', record_prefix='choices.')

# Make more important columns more visible

normalized_data = normalized_data[['choices.message.content', 'model', 
       'usage.completion_tokens', 'usage.prompt_tokens', 'usage.total_tokens', 'id', 
       'choices.finish_reason', 'choices.index', 'choices.logprobs', 
       'choices.message.role', 'choices.message.function_call', 'choices.message.tool_calls', 
       'created', 'object', 'service_tier', 'system_fingerprint'
       ]]

# Display

text = normalized_data["choices.message.content"].iloc[-1]
display_markdown(text, raw=True)
 

Prompt messages sent to OpenAI API:[{'role': 'user', 'content': 'Of course! Can you please provide me with the location or specific type of cuisine you are looking for so that I can make a tailored recommendation for you?'}]
Prompt messages sent to OpenAI API:[{'role': 'user', 'content': 'I am looking for a nice Italian restaurant in the downtown area.'}]


I would recommend checking out Osteria Morini located in the downtown area. They offer a delicious selection of traditional Italian dishes and have a cozy atmosphere perfect for a nice dinner. You can also try L'Artusi or Babbo for a more upscale dining experience. Enjoy your meal!